In [1]:
from __future__ import division
import sys
sys.path.insert(0, "~/.local/lib/python3.6/site-packages")

import sympy
from sympy import *
from sympy.physics.quantum import *

def express(a, b, name):
    sym = symbols(name)
    sol = solve(a-sym, b)
    assert len(sol) == 1
    return (sym, sol[0])


def frange(x, y, jump):
    i = 0
    curr = x + i*jump
    
    while curr < y:
        yield curr
        i += 1
        curr = x + i*jump
    

import gmpy2
import mpmath
gmpy2.get_context().precision = 10
mpmath.mp.dps = 10

import numpy
import scipy.linalg

In [2]:
# https://stackoverflow.com/q/59523322/1137334

from sympy.core.operations import AssocOp

def apply_ccr(expr, ccr, reverse=False):
    if not isinstance(expr, Basic):
        raise TypeError("The expression to simplify is not a sympy expression.")
        
    if not isinstance(ccr, Eq):
        if isinstance(ccr, Basic):
            ccr = Eq(ccr, 0)
        else:
            raise TypeError("The canonical commutation relation is not a sympy expression.")
    
    comm = None
    
    for node in preorder_traversal(ccr):
        if isinstance(node, Commutator):
            comm = node
            break
            
    if comm is None:
        raise ValueError("The cannonical commutation relation doesn not include a commutator.")
        
    solutions = solve(ccr, comm)
    
    if len(solutions) != 1:
        raise ValueError("There are more solutions to the cannonical commutation relation.")
        
    value = solutions[0]
    
    A = comm.args[0]
    B = comm.args[1]
    
    if reverse:
        (A, B) = (B, A)
        value = -value
    
    def is_expandable_pow_of(base, expr):
        return isinstance(expr, Pow) \
            and base == expr.args[0] \
            and isinstance(expr.args[1], Number) \
            and expr.args[1] >= 1
    
    
    def walk_tree(expr):
        if isinstance(expr, Number):
            return expr
        
        if not isinstance(expr, AssocOp) and not isinstance(expr, Function):
            return expr.copy()
        
        elif not isinstance(expr, Mul):
            return expr.func(*(walk_tree(node) for node in expr.args))
        
        else:
            args = [arg for arg in expr.args]
            
            for i in range(len(args)-1):
                x = args[i]
                y = args[i+1]
                
                if B == x and A == y:
                    args = args[0:i] + [A*B - value] + args[i+2:]
                    return walk_tree( Mul(*args).expand() )
                
                if B == x and is_expandable_pow_of(A, y):
                    ypow = Pow(A, y.args[1] - 1)
                    args = args[0:i] + [A*B - value, ypow] + args[i+2:]
                    return walk_tree( Mul(*args).expand() )
                
                if is_expandable_pow_of(B, x) and A == y:
                    xpow = Pow(B, x.args[1] - 1)
                    args = args[0:i] + [xpow, A*B - value] + args[i+2:]
                    return walk_tree( Mul(*args).expand() )
                
                if is_expandable_pow_of(B, x) and is_expandable_pow_of(A, y):
                    xpow = Pow(B, x.args[1] - 1)
                    ypow = Pow(A, y.args[1] - 1)
                    args = args[0:i] + [xpow, A*B - value, ypow] + args[i+2:]
                    return walk_tree( Mul(*args).expand() )
            
            return expr.copy()
            
    
    return walk_tree(expr)
   

Basic.apply_ccr = lambda self, ccr, reverse=False: apply_ccr(self, ccr, reverse)


In [3]:
s1_ = (1/sqrt(2)) * \
    Matrix([
        [0,1,0],
        [1,0,1],
        [0,1,0]
    ])
    
s2_ = (1/sqrt(2)) * \
    Matrix([
        [0,-I,0],
        [I,0,-I],
        [0, I,0]
    ])

s3_ = Matrix([
        [1,0, 0],
        [0,0, 0],
        [0,0,-1]
    ])

In [4]:
s1_*s2_

Matrix([
[I/2, 0, -I/2],
[  0, 0,    0],
[I/2, 0, -I/2]])

In [5]:
s2_*s1_

Matrix([
[-I/2, 0, -I/2],
[   0, 0,    0],
[ I/2, 0,  I/2]])

In [6]:
s2_*s3_

Matrix([
[          0, 0,           0],
[sqrt(2)*I/2, 0, sqrt(2)*I/2],
[          0, 0,           0]])

In [7]:
s3_*s2_

Matrix([
[0, -sqrt(2)*I/2, 0],
[0,            0, 0],
[0, -sqrt(2)*I/2, 0]])

In [8]:
s3_*s1_

Matrix([
[0,  sqrt(2)/2, 0],
[0,          0, 0],
[0, -sqrt(2)/2, 0]])

In [9]:
s1_*s3_

Matrix([
[        0, 0,          0],
[sqrt(2)/2, 0, -sqrt(2)/2],
[        0, 0,          0]])

In [10]:
s1_*s1_

Matrix([
[1/2, 0, 1/2],
[  0, 1,   0],
[1/2, 0, 1/2]])

In [11]:
s2_*s2_

Matrix([
[ 1/2, 0, -1/2],
[   0, 1,    0],
[-1/2, 0,  1/2]])

In [12]:
s3_*s3_

Matrix([
[1, 0, 0],
[0, 0, 0],
[0, 0, 1]])

In [13]:
n1, n2, n3 = symbols('n_1 n_2 n_3')
s1, s2, s3 = Operator('s_1'), Operator('s_2'), Operator('s_3')

ns = n1*s1 + n2*s2 + n3*s3
ns_ = simplify(n1*s1_ + n2*s2_ + n3*s3_)
ns_

Matrix([
[                    n_3, sqrt(2)*(n_1 - I*n_2)/2,                       0],
[sqrt(2)*(n_1 + I*n_2)/2,                       0, sqrt(2)*(n_1 - I*n_2)/2],
[                      0, sqrt(2)*(n_1 + I*n_2)/2,                    -n_3]])

In [14]:
simplify(ns_*ns_*ns_/(n1*n1 + n2*n2 + n3*n3))

Matrix([
[                    n_3, sqrt(2)*(n_1 - I*n_2)/2,                       0],
[sqrt(2)*(n_1 + I*n_2)/2,                       0, sqrt(2)*(n_1 - I*n_2)/2],
[                      0, sqrt(2)*(n_1 + I*n_2)/2,                    -n_3]])

In [15]:
simplify(ns_*ns_)

Matrix([
[n_1**2/2 + n_2**2/2 + n_3**2,  sqrt(2)*n_3*(n_1 - I*n_2)/2,           (n_1 - I*n_2)**2/2],
[ sqrt(2)*n_3*(n_1 + I*n_2)/2,              n_1**2 + n_2**2, sqrt(2)*n_3*(-n_1 + I*n_2)/2],
[          (n_1 + I*n_2)**2/2, -sqrt(2)*n_3*(n_1 + I*n_2)/2, n_1**2/2 + n_2**2/2 + n_3**2]])

In [16]:
E = Matrix.eye(3)
simplify(E + sinh(1)*ns_ + cosh(1)*ns_**2)

Matrix([
[n_3*sinh(1) + (2*n_3**2 + (n_1 - I*n_2)*(n_1 + I*n_2))*cosh(1)/2 + 1,  sqrt(2)*(n_1 - I*n_2)*(n_3*cosh(1) + sinh(1))/2,                                            (n_1 - I*n_2)**2*cosh(1)/2],
[                     sqrt(2)*(n_1 + I*n_2)*(n_3*cosh(1) + sinh(1))/2,          (n_1 - I*n_2)*(n_1 + I*n_2)*cosh(1) + 1,                      -sqrt(2)*(n_1 - I*n_2)*(n_3*cosh(1) - sinh(1))/2],
[                                          (n_1 + I*n_2)**2*cosh(1)/2, -sqrt(2)*(n_1 + I*n_2)*(n_3*cosh(1) - sinh(1))/2, -n_3*sinh(1) + (2*n_3**2 + (n_1 - I*n_2)*(n_1 + I*n_2))*cosh(1)/2 + 1]])

In [17]:
Matrix([1,2,3]).norm()

sqrt(14)

In [18]:
eye = Operator('1')
def expi(arg, mat):
    return eye + I*sin(arg)*mat + (cos(arg)-1)*mat**2

def refine(U):
    U = U.expand()
    U = U.apply_ccr(ccr1)
    U = U.apply_ccr(ccr2)
    U = U.apply_ccr(ccr3)
    U = U.subs(s1**3, s1).subs(s2**3,s2).subs(s3**3,s3)
    return U

t = symbols('t', real=True)
o, o0, o1 = symbols('omega, omega_0, omega_1', real=True)
O = symbols('Omega', real=True)
O_ = Matrix([-o1, 0, o-o0]).norm()

nsO = (1/O)*ns.subs(n1, -o1).subs(n2, 0).subs(n3, o-o0)

ccr1 = Eq(Commutator(s1,s2),I*s3)
ccr2 = Eq(Commutator(s2,s3),I*s1)
ccr3 = Eq(Commutator(s3,s1),I*s2)

U1 = expi(o*t, s3)
U2 = expi(-O*t,nsO)
U1 = U1.subs(s1,s1_).subs(s2,s2_).subs(s3,s3_).subs(eye, E)
U2 = U2.subs(s1,s1_).subs(s2,s2_).subs(s3,s3_).subs(eye, E)
U1 = simplify(U1)
U2 = simplify(U2)
U_ = U1*U2
U_ = simplify(U_)
U_

Matrix([
[(Omega**2 - I*Omega*(omega - omega_0)*sin(Omega*t) + (omega_1**2 + 2*(omega - omega_0)**2)*(cos(Omega*t) - 1)/2)*exp(I*omega*t)/Omega**2, sqrt(2)*omega_1*(I*Omega*sin(Omega*t) + (-omega + omega_0)*(cos(Omega*t) - 1))*exp(I*omega*t)/(2*Omega**2),                                                                                 omega_1**2*(cos(Omega*t) - 1)*exp(I*omega*t)/(2*Omega**2)],
[                                             sqrt(2)*omega_1*(I*Omega*sin(Omega*t) + (-omega + omega_0)*(cos(Omega*t) - 1))/(2*Omega**2),                                                        (Omega**2 + omega_1**2*(cos(Omega*t) - 1))/Omega**2,                                                sqrt(2)*omega_1*(I*Omega*sin(Omega*t) + (omega - omega_0)*(cos(Omega*t) - 1))/(2*Omega**2)],
[                                                                              omega_1**2*(cos(Omega*t) - 1)*exp(-I*omega*t)/(2*Omega**2), sqrt(2)*omega_1*(I*Omega*sin(Omega*t) + (omega - omega_0)*(cos(Omega*t) - 1))*

In [19]:
U1

Matrix([
[exp(I*omega*t), 0,               0],
[             0, 1,               0],
[             0, 0, exp(-I*omega*t)]])

In [20]:
U2

Matrix([
[(Omega**2 - I*Omega*(omega - omega_0)*sin(Omega*t) + (omega_1**2 + 2*(omega - omega_0)**2)*(cos(Omega*t) - 1)/2)/Omega**2, sqrt(2)*omega_1*(I*Omega*sin(Omega*t) + (-omega + omega_0)*(cos(Omega*t) - 1))/(2*Omega**2),                                                                                omega_1**2*(cos(Omega*t) - 1)/(2*Omega**2)],
[                              sqrt(2)*omega_1*(I*Omega*sin(Omega*t) + (-omega + omega_0)*(cos(Omega*t) - 1))/(2*Omega**2),                                         (Omega**2 + omega_1**2*(cos(Omega*t) - 1))/Omega**2,                                sqrt(2)*omega_1*(I*Omega*sin(Omega*t) + (omega - omega_0)*(cos(Omega*t) - 1))/(2*Omega**2)],
[                                                                               omega_1**2*(cos(Omega*t) - 1)/(2*Omega**2),  sqrt(2)*omega_1*(I*Omega*sin(Omega*t) + (omega - omega_0)*(cos(Omega*t) - 1))/(2*Omega**2), (Omega**2 + I*Omega*(omega - omega_0)*sin(Omega*t) + (omega_1**2 + 2*(omega - omega_0)**2)*(

In [21]:
print(latex(U1))

\left[\begin{matrix}e^{i \omega t} & 0 & 0\\0 & 1 & 0\\0 & 0 & e^{- i \omega t}\end{matrix}\right]


In [22]:
print(latex(simplify(U2*2*O**2)))

\left[\begin{matrix}2 \Omega^{2} - 2 i \Omega \left(\omega - \omega_{0}\right) \sin{\left(\Omega t \right)} + \left(\omega_{1}^{2} + 2 \left(\omega - \omega_{0}\right)^{2}\right) \left(\cos{\left(\Omega t \right)} - 1\right) & \sqrt{2} \omega_{1} \left(i \Omega \sin{\left(\Omega t \right)} + \left(- \omega + \omega_{0}\right) \left(\cos{\left(\Omega t \right)} - 1\right)\right) & \omega_{1}^{2} \left(\cos{\left(\Omega t \right)} - 1\right)\\\sqrt{2} \omega_{1} \left(i \Omega \sin{\left(\Omega t \right)} + \left(- \omega + \omega_{0}\right) \left(\cos{\left(\Omega t \right)} - 1\right)\right) & 2 \Omega^{2} + 2 \omega_{1}^{2} \left(\cos{\left(\Omega t \right)} - 1\right) & \sqrt{2} \omega_{1} \left(i \Omega \sin{\left(\Omega t \right)} + \left(\omega - \omega_{0}\right) \left(\cos{\left(\Omega t \right)} - 1\right)\right)\\\omega_{1}^{2} \left(\cos{\left(\Omega t \right)} - 1\right) & \sqrt{2} \omega_{1} \left(i \Omega \sin{\left(\Omega t \right)} + \left(\omega - \omega_{0}\right) \lef

In [23]:
print(latex(simplify(U_*2*O**2)))

\left[\begin{matrix}\left(2 \Omega^{2} - 2 i \Omega \left(\omega - \omega_{0}\right) \sin{\left(\Omega t \right)} + \left(\omega_{1}^{2} + 2 \left(\omega - \omega_{0}\right)^{2}\right) \left(\cos{\left(\Omega t \right)} - 1\right)\right) e^{i \omega t} & \sqrt{2} \omega_{1} \left(i \Omega \sin{\left(\Omega t \right)} + \left(- \omega + \omega_{0}\right) \left(\cos{\left(\Omega t \right)} - 1\right)\right) e^{i \omega t} & \omega_{1}^{2} \left(\cos{\left(\Omega t \right)} - 1\right) e^{i \omega t}\\\sqrt{2} \omega_{1} \left(i \Omega \sin{\left(\Omega t \right)} + \left(- \omega + \omega_{0}\right) \left(\cos{\left(\Omega t \right)} - 1\right)\right) & 2 \Omega^{2} + 2 \omega_{1}^{2} \left(\cos{\left(\Omega t \right)} - 1\right) & \sqrt{2} \omega_{1} \left(i \Omega \sin{\left(\Omega t \right)} + \left(\omega - \omega_{0}\right) \left(\cos{\left(\Omega t \right)} - 1\right)\right)\\\omega_{1}^{2} \left(\cos{\left(\Omega t \right)} - 1\right) e^{- i \omega t} & \sqrt{2} \omega_{1} \left(i 

In [24]:
U_inv = U_.adjoint()
U_inv

Matrix([
[(Omega**2 + I*Omega*(omega - omega_0)*sin(Omega*t) + (omega_1**2 + 2*(omega - omega_0)**2)*(cos(Omega*t) - 1)/2)*exp(-I*omega*t)/Omega**2, sqrt(2)*omega_1*(-I*Omega*sin(Omega*t) + (-omega + omega_0)*(cos(Omega*t) - 1))/(2*Omega**2),                                                                                omega_1**2*(cos(Omega*t) - 1)*exp(I*omega*t)/(2*Omega**2)],
[                             sqrt(2)*omega_1*(-I*Omega*sin(Omega*t) + (-omega + omega_0)*(cos(Omega*t) - 1))*exp(-I*omega*t)/(2*Omega**2),                                          (Omega**2 + omega_1**2*(cos(Omega*t) - 1))/Omega**2,                               sqrt(2)*omega_1*(-I*Omega*sin(Omega*t) + (omega - omega_0)*(cos(Omega*t) - 1))*exp(I*omega*t)/(2*Omega**2)],
[                                                                               omega_1**2*(cos(Omega*t) - 1)*exp(-I*omega*t)/(2*Omega**2),  sqrt(2)*omega_1*(-I*Omega*sin(Omega*t) + (omega - omega_0)*(cos(Omega*t) - 1))/(2*Omega**2), (Omega**2 -

In [79]:
from sympy.simplify import trigsimp
t0 = symbols('t_0', real=True)

U1 = exp(I*o*t*s3)
U2 = exp(-I*O*(t-t0)*nsO)
U3 = exp(-I*o*t0*s3)

Utt0 = U1*U2*U3

U1 = expi(o*t, s3)
U2 = expi(-O*(t-t0),nsO)
U3 = expi(-o*t0, s3)

U1, U2, U3 = [uuu.subs(s1,s1_).subs(s2,s2_).subs(s3,s3_).subs(eye, E) for uuu in (U1,U2,U3)]
U1, U2, U3 = [simplify(uuu) for uuu in (U1,U2,U3)]

Utt0_ = U1*U2*U3
Utt0_ = Utt0_.subs(O_**2, O**2)
Utt0_ = simplify(Utt0_)
Utt0_

Matrix([
[(Omega**2 - I*Omega*(omega - omega_0)*sin(Omega*(t - t_0)) + (omega_1**2 + 2*(omega - omega_0)**2)*(cos(Omega*(t - t_0)) - 1)/2)*exp(I*omega*(t - t_0))/Omega**2, sqrt(2)*omega_1*(I*Omega*sin(Omega*(t - t_0)) + (-omega + omega_0)*(cos(Omega*(t - t_0)) - 1))*exp(I*omega*t)/(2*Omega**2),                                                                                         omega_1**2*(cos(Omega*(t - t_0)) - 1)*exp(I*omega*(t + t_0))/(2*Omega**2)],
[                                   sqrt(2)*omega_1*(I*Omega*sin(Omega*(t - t_0)) + (-omega + omega_0)*(cos(Omega*(t - t_0)) - 1))*exp(-I*omega*t_0)/(2*Omega**2),                                                                (Omega**2 + omega_1**2*(cos(Omega*(t - t_0)) - 1))/Omega**2,                                       sqrt(2)*omega_1*(I*Omega*sin(Omega*(t - t_0)) + (omega - omega_0)*(cos(Omega*(t - t_0)) - 1))*exp(I*omega*t_0)/(2*Omega**2)],
[                                                                                      om

In [64]:
print(latex(simplify(Utt0_*(2*O**2))))

2 \left(\left(\cos{\left(\omega t \right)} - 1\right) s_{3}^{2} + i \sin{\left(\omega t \right)} s_{3} + 1\right) \left(\Omega^{2} \cdot 1 - i \Omega \sin{\left(\Omega \left(t - t_{0}\right) \right)} \left(- \omega_{1} s_{1} + \left(\omega - \omega_{0}\right) s_{3}\right) + \left(\cos{\left(\Omega \left(t - t_{0}\right) \right)} - 1\right) \left(- \omega_{1} s_{1} + \left(\omega - \omega_{0}\right) s_{3}\right)^{2}\right) \left(\left(\cos{\left(\omega t_{0} \right)} - 1\right) s_{3}^{2} - i \sin{\left(\omega t_{0} \right)} s_{3} + 1\right)


In [66]:
tau, T = symbols('tau T', real=True)

U1 = expi(o*(tau+T), s3)
U2 = expi(-(o-o0)*T,nsO.subs(O,o-o0))
U3 = expi(-o*tau, s3)

U0 = U1*U2*U3

U1, U2, U3 = [uuu.subs(s1,s1_).subs(s2,s2_).subs(s3,s3_).subs(eye, E) for uuu in (U1,U2,U3)]
U1, U2, U3 = [simplify(uuu) for uuu in (U1,U2,U3)]

U0_ = U1*U2*U3
U0_ = simplify(U0_)

U0rez_ = U1*U3
U0rez_ = simplify(U0rez_)
U0_

Matrix([
[(-2*(omega - omega_0)**2*(I*sin(T*(omega - omega_0)) - 1) + (omega_1**2 + 2*(omega - omega_0)**2)*(cos(T*(omega - omega_0)) - 1))*exp(I*T*omega)/(2*(omega - omega_0)**2),     sqrt(2)*omega_1*(exp(I*T*omega) - exp(I*T*omega_0))*exp(I*omega*tau)/(2*(omega - omega_0)),                                                                               omega_1**2*(cos(T*(omega - omega_0)) - 1)*exp(I*omega*(T + 2*tau))/(2*(omega - omega_0)**2)],
[                                                         sqrt(2)*omega_1*(exp(I*T*(omega - omega_0)) - 1)*exp(-I*(T*(omega - omega_0) + omega*tau))/(2*(omega - omega_0)),        (omega_1**2*(cos(T*(omega - omega_0)) - 1) + (omega - omega_0)**2)/(omega - omega_0)**2,                                                                                   sqrt(2)*omega_1*(exp(I*T*(omega - omega_0)) - 1)*exp(I*omega*tau)/(2*omega - 2*omega_0)],
[                                                                             omega_1**2*(cos(T*(omega - omega_

In [67]:
print(latex(simplify(U0_*2*(o-o0)**2)))

\left[\begin{matrix}\left(- 2 \left(\omega - \omega_{0}\right)^{2} \left(i \sin{\left(T \left(\omega - \omega_{0}\right) \right)} - 1\right) + \left(\omega_{1}^{2} + 2 \left(\omega - \omega_{0}\right)^{2}\right) \left(\cos{\left(T \left(\omega - \omega_{0}\right) \right)} - 1\right)\right) e^{i T \omega} & \sqrt{2} \omega_{1} \left(\omega - \omega_{0}\right) \left(e^{i T \omega} - e^{i T \omega_{0}}\right) e^{i \omega \tau} & \omega_{1}^{2} \left(\cos{\left(T \left(\omega - \omega_{0}\right) \right)} - 1\right) e^{i \omega \left(T + 2 \tau\right)}\\\sqrt{2} \omega_{1} \left(\omega - \omega_{0}\right) \left(e^{i T \left(\omega - \omega_{0}\right)} - 1\right) e^{- i \left(T \left(\omega - \omega_{0}\right) + \omega \tau\right)} & 2 \omega_{1}^{2} \left(\cos{\left(T \left(\omega - \omega_{0}\right) \right)} - 1\right) + 2 \left(\omega - \omega_{0}\right)^{2} & \sqrt{2} \omega_{1} \left(\omega - \omega_{0}\right) \left(e^{i T \left(\omega - \omega_{0}\right)} - 1\right) e^{i \omega \tau}\\

In [68]:
U0rez_

Matrix([
[exp(I*T*omega), 0,               0],
[             0, 1,               0],
[             0, 0, exp(-I*T*omega)]])

In [69]:
print(latex(U0rez_))

\left[\begin{matrix}e^{i T \omega} & 0 & 0\\0 & 1 & 0\\0 & 0 & e^{- i T \omega}\end{matrix}\right]


In [81]:
Utt0rez = Utt0.subs(O,O_).subs(o,o0)
U1 = Utt0rez.subs(t,2*tau+T).subs(t0,tau+T)
U2 = Utt0rez.subs(t,tau+T).subs(t0,tau).subs(o1,0)
U3 = Utt0rez.subs(t,tau).subs(t0,0)

UFrez = U1*U2*U3
UFrez

exp(I*omega_0*(T + 2*tau)*s_3)*exp(2*I*omega_1*tau*s_1)

In [92]:
U1 = expi(o0 * (T+2*tau), s3)
U2 = expi(2* o1 * tau, s1)

U1, U2 = [uuu.subs(s1,s1_).subs(s2,s2_).subs(s3,s3_).subs(eye, E) for uuu in (U1,U2)]

UFrez_ = U1*U2
UFrez_ = simplify(UFrez_)
UFrez_ = trigsimp(UFrez_)
UFrez_

Matrix([
[ (cos(2*omega_1*tau) + 1)*exp(I*omega_0*(T + 2*tau))/2, sqrt(2)*(-sin(omega_0*(T + 2*tau)) + I*cos(omega_0*(T + 2*tau)))*sin(2*omega_1*tau)/2,  (cos(2*omega_1*tau) - 1)*exp(I*omega_0*(T + 2*tau))/2],
[                        sqrt(2)*I*sin(2*omega_1*tau)/2,                                                                    cos(2*omega_1*tau),                         sqrt(2)*I*sin(2*omega_1*tau)/2],
[(cos(2*omega_1*tau) - 1)*exp(-I*omega_0*(T + 2*tau))/2,  sqrt(2)*(sin(omega_0*(T + 2*tau)) + I*cos(omega_0*(T + 2*tau)))*sin(2*omega_1*tau)/2, (cos(2*omega_1*tau) + 1)*exp(-I*omega_0*(T + 2*tau))/2]])

In [94]:
print(latex(UFrez_*2))

\left[\begin{matrix}\left(\cos{\left(2 \omega_{1} \tau \right)} + 1\right) e^{i \omega_{0} \left(T + 2 \tau\right)} & \sqrt{2} \left(- \sin{\left(\omega_{0} \left(T + 2 \tau\right) \right)} + i \cos{\left(\omega_{0} \left(T + 2 \tau\right) \right)}\right) \sin{\left(2 \omega_{1} \tau \right)} & \left(\cos{\left(2 \omega_{1} \tau \right)} - 1\right) e^{i \omega_{0} \left(T + 2 \tau\right)}\\\sqrt{2} i \sin{\left(2 \omega_{1} \tau \right)} & 2 \cos{\left(2 \omega_{1} \tau \right)} & \sqrt{2} i \sin{\left(2 \omega_{1} \tau \right)}\\\left(\cos{\left(2 \omega_{1} \tau \right)} - 1\right) e^{- i \omega_{0} \left(T + 2 \tau\right)} & \sqrt{2} \left(\sin{\left(\omega_{0} \left(T + 2 \tau\right) \right)} + i \cos{\left(\omega_{0} \left(T + 2 \tau\right) \right)}\right) \sin{\left(2 \omega_{1} \tau \right)} & \left(\cos{\left(2 \omega_{1} \tau \right)} + 1\right) e^{- i \omega_{0} \left(T + 2 \tau\right)}\end{matrix}\right]


In [96]:
Prez = UFrez_.applyfunc(lambda x: abs(x)**2)
Prez = simplify(Prez)
Prez

Matrix([
[    cos(omega_1*tau)**4, sin(2*omega_1*tau)**2/2,     sin(omega_1*tau)**4],
[sin(2*omega_1*tau)**2/2,   cos(2*omega_1*tau)**2, sin(2*omega_1*tau)**2/2],
[    sin(omega_1*tau)**4, sin(2*omega_1*tau)**2/2,     cos(omega_1*tau)**4]])

In [97]:
print(latex(Prez))

\left[\begin{matrix}\cos^{4}{\left(\omega_{1} \tau \right)} & \frac{\sin^{2}{\left(2 \omega_{1} \tau \right)}}{2} & \sin^{4}{\left(\omega_{1} \tau \right)}\\\frac{\sin^{2}{\left(2 \omega_{1} \tau \right)}}{2} & \cos^{2}{\left(2 \omega_{1} \tau \right)} & \frac{\sin^{2}{\left(2 \omega_{1} \tau \right)}}{2}\\\sin^{4}{\left(\omega_{1} \tau \right)} & \frac{\sin^{2}{\left(2 \omega_{1} \tau \right)}}{2} & \cos^{4}{\left(\omega_{1} \tau \right)}\end{matrix}\right]
